In [13]:
import os
import numpy as np
import pickle
import random
import collections
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
import datetime  
from datetime import date 
import calendar 
  
def findDay(date): 
    year, month, day = (int(i) for i in date.split('-'))     
    born = datetime.date(year, month, day)
    return born.strftime("%A") 
  
# Driver program 
date = '2019-03-02'
print(findDay(date)) 

Saturday


In [14]:
df = pd.read_csv('train.csv')

In [109]:
dfv = pd.read_csv('dev.csv')

In [73]:
df.iloc[0]['date']

'2014-12-08'

In [74]:
def date1(x):
    return findDay(x['date'])

In [75]:
date1(df.iloc[0])

'Monday'

In [76]:
def read_data(file):
    '''
    Read each file into a list of strings.
    Example:
    ["it's", 'a', 'curious', 'thing', "i've", 'found', 'that', 'when', 'willis', 'is', 'not', 'called', 'on',
    ...'to', 'carry', 'the', 'whole', 'movie', "he's", 'much', 'better', 'and', 'so', 'is', 'the', 'movie']
    '''
    f = open(file)
    lines = f.read().split(' ')
    symbols = '${}()[].,:;+-*/&|<>=~" '
    words = map(lambda Element: Element.translate(str.maketrans("", "", symbols)).strip(), lines)
    words = filter(None, words)
    return list(words)

In [77]:
class reviewInstance:

    def __init__(self,review):
 
        self.review=review['review']
        self.wordic=[]
        self.weekday=findDay(review['date'])
        self.count=0
        self.label=review['label']
        self.rate=review['rating']
        self.exid=review['ex_id']
        self.userid=review['user_id']
        self.prodid=review['prod_id']
        
        lines = self.review.split(' ')
        symbols = '${}()[].,:;+-*/&|<>=~" '
        words = map(lambda Element: Element.translate(str.maketrans("", "", symbols)).strip(), lines)
        words = filter(None, words)
        self.rlist=list(words)
        self.count=len(self.rlist)

        
    def construct_word_dict(self, stop_word=None, count_words=None):
        """
        Description
        ===========
        Count words in word_list to transform to a dict {word: word_count}
        
        Input
        =====
        stop_word: list
        Words you hope to filter, not included in dict, default set to None
        
        count_words: list
        Words you hope to keep in the count_dict, if set to None, will keep all the words
        """
        wordlist=self.rlist
       
        r = collections.Counter()
        if count_words == None:
            if stop_word:
                for word in wordlist:
                    if word not in stop_word:

                        r[word] += 1
            else: 
                for word in wordlist:
                    r[word] += 1
                    
        else:
            if stop_word:
                for word in wordlist:
                    if word not in stop_word:

                        r[word] += 1
            else: 

                for word in wordlist:
                    if (word not in stop_word) and (word in count_words):

                        r[word] += 1
        self.wordic=r
    
        
        

            

In [78]:
x=reviewInstance(df.iloc[2])

In [128]:
df

,ex_id,user_id,prod_id,rating,label,date,review
0,0,923,0,3.0,1,2014-12-08,The food at snack is a selection of popular Gr...
1,1,924,0,3.0,1,2013-05-16,This little place in Soho is wonderful. I had ...
2,2,925,0,4.0,1,2013-07-01,ordered lunch for 15 from Snack last Friday. ...
3,3,926,0,4.0,1,2011-07-28,This is a beautiful quaint little restaurant o...
4,4,927,0,4.0,1,2010-11-01,Snack is great place for a casual sit down lu...
...,...,...,...,...,...,...,...
250869,358950,14671,349,5.0,0,2014-02-08,Made a reservation for an early dinner Saturda...
250870,358951,3356,349,5.0,0,2014-02-07,"Emily is like Franny's Marco, but with warmth ..."
250871,358953,116424,349,5.0,0,2014-01-31,Can't say enough good things about this place....
250872,358954,161147,349,5.0,0,2014-01-30,"Had a great dinner here- fantastic pizza, the ..."


In [113]:
n=len(df)

In [114]:
n

250874

In [81]:
X=[]
for i in range(n):
    x=reviewInstance(df.iloc[i])
    x.construct_word_dict()
    X.append(x)


In [115]:
n2=len(dfv)

In [116]:

X2=[]
for i in range(n2):
    x=reviewInstance(dfv.iloc[i])
    x.construct_word_dict()
    X2.append(x)




In [85]:
with open('review_train.pkl', 'wb') as f:
        pickle.dump(X, f)

In [92]:
with open('review_train.pkl', "rb") as f:
    review_t=pickle.load(f)

In [93]:
review_t[6].exid

7

In [117]:
with open('review_val.pkl', 'wb') as f:
        pickle.dump(X2, f)

In [118]:
with open('review_val.pkl', "rb") as f:
    review_v=pickle.load(f)

In [94]:
n=len(review_t)

In [95]:
n

250874

In [96]:
weekday1=0
weekend1=0
weekday0=0
weekend0=0
for i in range(n):
    if review_t[i].label==1:
        if review_t[i].weekday == 'Sunday' or review_t[i].weekday == 'Saturday' :
            
            weekend1=weekend1+1
        else:
            weekday1=weekday1+1
    else:
        if review_t[i].weekday == 'Sunday' or review_t[i].weekday == 'Saturday':
            weekend0=weekend0+1
        else:
            weekday0=weekday0+1
        
        
    

In [97]:
weekend1/(weekday1+weekend1)

0.2698787714473837

In [98]:
weekend0/(weekend0+weekday0)

0.29582990824465133

In [99]:
2/7

0.2857142857142857

Looks like fake reviews are more likely to be sent during weekday 3% higher than real reviews

In [100]:
def dotProduct(d1, d2):
    """
    @param dict d1: a feature vector represented by a mapping from a feature (string) to a weight (float).
    @param dict d2: same as d1
    @return float: the dot product between d1 and d2
    """
    if len(d1) < len(d2):
        return dotProduct(d2, d1)
    else:
        return sum(d1.get(f, 0) * v for f, v in d2.items())

def increment(d1, scale, d2):
    """
    Implements d1 += scale * d2 for sparse vectors.
    @param dict d1: the feature vector which is mutated.
    @param float scale
    @param dict d2: a feature vector.

    NOTE: This function does not return anything, but rather
    increments d1 in place. We do this because it is much faster to
    change elements of d1 in place than to build a new dictionary and
    return it.
    """
    for f, v in d2.items():
        d1[f] = d1.get(f, 0) + v * scale

In [129]:
def fastpegasos(review, max_epoch, lam, watch_list=None, grad_checking=False):
    """
    Description
    ==========
    Implementat
    ion of Pegasos Algorithm 

    Input
    =====
    review_list: list of reviewInstance's
    list of objects with labels and encoded input from reviews

    max_epoch: int 
    stopping condition

    lam: float 
    regularization parameter

    watch_list: list or reviewInstance's
    passed to accuracy_percent or magnitude_compare; default None

    grad_checking: bool 
    numerical test of gradient of svm objective

    Output
    ======
    weights
    """
    
    #Initialization
    s=1
    w = collections.Counter()
    epoch = 0
    t = 1
    n = len(review)
    
    

    #Loop
    # Use the util.increment and util.dotProduct functions in update
    while epoch < max_epoch:
        #print("epoch=", epoch)
        for i in range(n):
            y = review[i].label*2-1
            
         

                
            t = t+1
            ita = 1/(t*lam)
            s=(1-ita*lam)*s
            
            
            
            
            if y*dotProduct(w, review[i].wordic) < 1:
                
                
                
                #Implements d1 += scale * d2 for sparse vectors.
                
                increment(w, ita*y/s,review[i].wordic)
            
        
        epoch=epoch+1
    increment(w,s-1,w)
    
        
    return w

In [138]:
review_t[10].label

1

In [131]:
w=fastpegasos(review_t, 5 , 1, watch_list=None, grad_checking=False)

In [136]:
def rate(w,review):

    n=len(review)
    cor=0
    wro=0
    for i in range(n):
        if (review[i].label*2-1)*np.sign(dotProduct(w,review[i].wordic)) > 0:
            cor = cor + 1
        else:
            wro = wro + 1
    return wro/(cor+wro)

In [139]:
rate(w,review_v)

0.11414889470460493